In [ ]:
import mujoco
import mujoco.viewer as viewer
import numpy as np
import time
import maze
import fogVisualize

'''
generates a maze + its solution using maze.py, converts the
maze structure into a format usable by MuJoCo world builder, 
converts the solution path into movement instructions, and prints
a small status summary
'''
print("Generating maze from maze.py...")
grid, path, visited_nodes, expansion_order, removed_walls_david = maze.generate_and_solve_maze()
remove_walls = maze.convert_david_to_mujoco_format(removed_walls_david, maze.COLS)
instructions = maze.generate_move_instructions(path)
print(f"✓ Maze generated with {len(remove_walls)} walls and {len(instructions)} steps")

# --------------------
# Maze parameters
# --------------------
cell_size = 0.15
wall_thickness = 0.01
wall_height = 0.1
maze_size = 30 * cell_size / 2  # Changed from 6 to 30

def should_remove_wall(cell1, cell2):
    return (cell1, cell2) in remove_walls or (cell2, cell1) in remove_walls

# --------------------
# Scaling and offsets
# --------------------
y_shift = 0


# --------------------
# Build walls
# --------------------
h_walls = []
for row in range(29):  # 29 horizontal walls (30 rows needs 29 dividers)
    for col in range(30):  # 30 columns
        cell_above = row * 30 + col + 1
        cell_below = (row + 1) * 30 + col + 1
        if not should_remove_wall(cell_above, cell_below):
            y_pos = maze_size - (row + 1) * cell_size + y_shift
            x_pos = -maze_size + col * cell_size + cell_size / 2
            h_walls.append(f'<body name="h_{row}_{col}" pos="{x_pos} {y_pos} 0">'
                           f'<geom type="box" size="{cell_size/2  + wall_thickness/2} {wall_thickness/2} {wall_height}" '
                           f'rgba="0.7 0.7 0.7 1"/></body>')

v_walls = []
for col in range(29):  # 29 vertical walls (30 cols needs 29 dividers)
    for row in range(30):  # 30 rows
        cell_left = row * 30 + col + 1
        cell_right = row * 30 + col + 2
        if not should_remove_wall(cell_left, cell_right):
            x_pos = -maze_size + (col + 1) * cell_size
            y_pos = maze_size - row * cell_size - cell_size / 2 + y_shift
            v_walls.append(f'<body name="v_{row}_{col}" pos="{x_pos} {y_pos} 0">'
                           f'<geom type="box" size="{wall_thickness/2} {cell_size/2 + wall_thickness/2} {wall_height}" '
                           f'rgba="0.7 0.7 0.7 1"/></body>')

# Outer walls
outer_walls = f'''
<body name="wall_top" pos="0 {maze_size + y_shift} 0">
    <geom type="box" size="{maze_size + wall_thickness/2} {wall_thickness/2} {wall_height}" rgba="0.7 0.7 0.7 1"/>
</body>

<body name="wall_bottom" pos="0 {-maze_size + y_shift} 0">
    <geom type="box" size="{maze_size + wall_thickness/2} {wall_thickness/2} {wall_height}" rgba="0.7 0.7 0.7 1"/>
</body>

<body name="wall_left" pos="{-maze_size} {y_shift} 0">
    <geom type="box" size="{wall_thickness/2} {maze_size + wall_thickness/2} {wall_height}" rgba="0.7 0.7 0.7 1"/>
</body>

<body name="wall_right" pos="{maze_size} {y_shift} 0">
    <geom type="box" size="{wall_thickness/2} {maze_size + wall_thickness/2} {wall_height}" rgba="0.7 0.7 0.7 1"/>
</body>
'''

maze_section = outer_walls + chr(10).join(h_walls) + chr(10).join(v_walls)

# Defines a rectangular region inside the maze
OBS_R_START = 10
OBS_R_END = 20
OBS_C_START = 10
OBS_C_END = 20

obstacle_blocks = []
for r in range(OBS_R_START, OBS_R_END):
    for c in range(OBS_C_START, OBS_C_END):
        # Converts each (row, column) to world coordinates
        x_pos = -maze_size + c * cell_size + cell_size / 2
        y_pos = maze_size - r * cell_size - cell_size / 2 + y_shift
        
        # Creates a MuJoCo <body> with a box geom at that position
        obstacle_blocks.append(
            f'<body name="obs_{r}_{c}" pos="{x_pos} {y_pos} 0">'
            f'<geom type="box" size="{cell_size/2} {cell_size/2} {wall_height}" '
            f'rgba="0.2 0.2 0.2 1"/></body>'
        )

# Update maze_section to include obstacles
maze_section = outer_walls + chr(10).join(h_walls) + chr(10).join(v_walls) + chr(10).join(obstacle_blocks)

In [ ]:
## Robotic arm model: 4DOF

# --------------------
# Robot & ball parameters
# --------------------
l1, l2, l3 = 0.6, 2.5, 2.5
base_pos_x, base_pos_y, base_pos_z = 0, 0, wall_height + wall_thickness
ball_pos_x = -maze_size + 0 * cell_size + cell_size / 2  # Col 0
ball_pos_y = -(-maze_size + (30-1) * cell_size + cell_size / 2 + y_shift)  # Row 29 (bottom)
ball_pos_z = 0
ball_radius = 0.03

# TODO: Tune these damping values. They control the "stiffness" of each joint
# Be careful of kp4; I would not touch this. This is the end effector joint.
# For some reason, higher values make the entire arm go crazy. 
kp1, kp2, kp3, kp4 = 500, 1000, 1000, 50
grav = -9.81
prong_size=0.05
prong_tip_size=-ball_radius
prong_angle=np.pi/4
prong_tip_angle=np.pi/2+prong_angle

# --------------------
# Full Mujoco XML
# --------------------
robot_4dof = f'''
<mujoco model="3dof_arm">
    <compiler angle="radian" coordinate="local" meshdir="assets" autolimits="true"/>
    <option gravity="0 0 {grav}"/>

    <asset>
        <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3" rgb2=".2 .3 .4" width="300" height="300"/>
        <material name="grid" texture="grid" texrepeat="8 8" reflectance=".2"/>
        <mesh name="ee" file="ee1.STL" scale="0.05 0.05 0.05"/>
    </asset>

    <worldbody>
        <geom size="4 4 0.01" type="plane" material="grid"/>
        <light pos="0 0 1"/>

        <!-- Base and Arm -->
        <body name="base" pos="{base_pos_x} {base_pos_y} {base_pos_z}">
            <body name="joint1" pos="0 0 0">
                <joint name="joint1_hinge" type="hinge" axis="0 0 1"/>
                <geom type="sphere" size="0.05" rgba="1 0 0 1" contype="1" conaffinity="0"/>
                <geom type="cylinder" fromto="0 0 0 0 0 {l1}" size="0.02" rgba="0 1 0 1" contype="1" conaffinity="0"/>
                
                <body name="joint2" pos="0 0 {l1}" gravcomp="1">
                    <joint name="joint2_hinge" type="hinge" axis="0 1 0"/>
                    <geom type="sphere" size="0.05" rgba="1 0 0 1"/>
                    <geom type="cylinder" fromto="0 0 0 0 0 {l2}" size="0.02" rgba="0 1 0 1"/>
                    
                    <body name="joint3" pos="0 0 {l2}" gravcomp="1">
                        <joint name="joint3_hinge" type="hinge" axis="0 1 0"/>
                        <geom type="sphere" size="0.05" rgba="1 0 0 1"/>
                        <geom type="cylinder" fromto="0 0 0 0 0 {l3}" size="0.02" rgba="0 1 0 1"/>

                        <body name="ee" pos="0 0 {l3}" gravcomp="1">
                            <joint name="joint4_hinge" type="hinge" axis="0 1 0" actuatorgravcomp="true"/>
                            <geom type="sphere" size="0.02" rgba="1 1 1 1"/>

                            <!-- prong 0 -->
                            <body name="prong_rot_0" pos="0 0 0" euler="0 0 0">
                                <body name="prong_0" pos="0 0 0" euler="{prong_angle} 0 0">
                                    <geom  type="capsule" fromto="0 0 0 0 0 {prong_size}" size="0.01" rgba="0.8 0.8 0.8 1"/>
                                    <body name="prong_0_tip" pos="0 0 {prong_size}" euler="{prong_tip_angle} 0 0">
                                        <geom type="capsule" fromto="0 0 0 0 0 {prong_tip_size}" size="0.007" rgba="0.7 0.7 0.7 1"/>
                                    </body>
                                </body>
                            </body>

                            <!-- prong 1 -->
                            <body name="prong_rot_1" pos="0 0 0" euler="0 0 0.785398">
                                <body name="prong_1" pos="0 0 0" euler="{prong_angle} 0 0">
                                    <geom  type="capsule" fromto="0 0 0 0 0 {prong_size}" size="0.01" rgba="0.8 0.8 0.8 1"/>
                                    <body name="prong_1_tip" pos="0 0 {prong_size}" euler="{prong_tip_angle} 0 0">
                                        <geom type="capsule" fromto="0 0 0 0 0 {prong_tip_size}" size="0.007" rgba="0.7 0.7 0.7 1"/>
                                    </body>
                                </body>
                            </body>

                            <!-- prong 2 -->
                            <body name="prong_rot_2" pos="0 0 0" euler="0 0 1.570796">
                                <body name="prong_2" pos="0 0 0" euler="{prong_angle} 0 0">
                                    <geom type="capsule" fromto="0 0 0 0 0 {prong_size}" size="0.01"/>
                                    <body name="prong_2_tip" pos="0 0 {prong_size}" euler="{prong_tip_angle} 0 0">
                                        <geom type="capsule" fromto="0 0 0 0 0 {prong_tip_size}" size="0.007"/>
                                    </body>
                                </body>
                            </body>

                            <!-- prong 3 -->
                            <body name="prong_rot_3" pos="0 0 0" euler="0 0 2.356194">
                                <body name="prong_3" pos="0 0 0" euler="{prong_angle} 0 0">
                                    <geom type="capsule" fromto="0 0 0 0 0 {prong_size}" size="0.01"/>
                                    <body name="prong_3_tip" pos="0 0 {prong_size}" euler="{prong_tip_angle} 0 0">
                                        <geom type="capsule" fromto="0 0 0 0 0 {prong_tip_size}" size="0.007"/>
                                    </body>
                                </body>
                            </body>

                            <!-- prong 4 -->
                            <body name="prong_rot_4" pos="0 0 0" euler="0 0 3.141592">
                                <body name="prong_4" pos="0 0 0" euler="{prong_angle} 0 0">
                                    <geom type="capsule" fromto="0 0 0 0 0 {prong_size}" size="0.01"/>
                                    <body name="prong_4_tip" pos="0 0 {prong_size}" euler="{prong_tip_angle} 0 0">
                                        <geom type="capsule" fromto="0 0 0 0 0 {prong_tip_size}" size="0.007"/>
                                    </body>
                                </body>
                            </body>

                            <!-- prong 5 -->
                            <body name="prong_rot_5" pos="0 0 0" euler="0 0 3.926990">
                                <body name="prong_5" pos="0 0 0" euler="{prong_angle} 0 0">
                                    <geom type="capsule" fromto="0 0 0 0 0 {prong_size}" size="0.01"/>
                                    <body name="prong_5_tip" pos="0 0 {prong_size}" euler="{prong_tip_angle} 0 0">
                                        <geom type="capsule" fromto="0 0 0 0 0 {prong_tip_size}" size="0.007"/>
                                    </body>
                                </body>
                            </body>

                            <!-- prong 6 -->
                            <body name="prong_rot_6" pos="0 0 0" euler="0 0 4.712388">
                                <body name="prong_6" pos="0 0 0" euler="{prong_angle} 0 0">
                                    <geom type="capsule" fromto="0 0 0 0 0 {prong_size}" size="0.01"/>
                                    <body name="prong_6_tip" pos="0 0 {prong_size}" euler="{prong_tip_angle} 0 0">
                                        <geom type="capsule" fromto="0 0 0 0 0 {prong_tip_size}" size="0.007"/>
                                    </body>
                                </body>
                            </body>

                            <!-- prong 7 -->
                            <body name="prong_rot_7" pos="0 0 0" euler="0 0 5.497787">
                                <body name="prong_7" pos="0 0 0" euler="{prong_angle} 0 0">
                                    <geom type="capsule" fromto="0 0 0 0 0 {prong_size}" size="0.01"/>
                                    <body name="prong_7_tip" pos="0 0 {prong_size}" euler="{prong_tip_angle} 0 0">
                                        <geom type="capsule" fromto="0 0 0 0 0 {prong_tip_size}" size="0.007"/>
                                    </body>
                                </body>
                            </body>
                        </body>
                    </body>
                </body>
            </body>
        </body>

        <!-- Ball -->
        <body name="ball" pos="{ball_pos_x} {ball_pos_y} {ball_pos_z}">
            <geom type="sphere" size="{ball_radius}" rgba="1 1 0 1"/>
            <joint type="free"/>
        </body>

        <!-- Maze -->
        <body name="maze" pos="0 0 0">
            {maze_section}
        </body>

    </worldbody>

    <actuator>
        <position joint="joint1_hinge" kp="{kp1}" dampratio="3"/>
        <position joint="joint2_hinge" kp="{kp2}" dampratio="1"/>
        <position joint="joint3_hinge" kp="{kp3}" dampratio="1.5"/>
        <position joint="joint4_hinge" kp="{kp4}" dampratio="0.5"/>
    </actuator>
</mujoco>
'''

In [ ]:
## Robotic arm model: 5DOF

# --------------------
# Robot & ball parameters
# --------------------
l1, l2, l3, l4 = 0.3, 1.5, 1.5, 1.5
base_pos_x, base_pos_y, base_pos_z = 0, 0, wall_height + wall_thickness
ball_pos_x = -maze_size + 0 * cell_size + cell_size / 2  # Col 0
ball_pos_y = -(-maze_size + (30-1) * cell_size + cell_size / 2 + y_shift)  # Row 29 (bottom)
ball_pos_z = 0
ball_radius = 0.03

# TODO: Tune these damping values. They control the "stiffness" of each joint
# Be careful of kp4; I would not touch this. This is the end effector joint.
# For some reason, higher values make the entire arm go crazy. 
kp1, kp2, kp3, kp4, kpEE = 1000, 1000, 1000, 1000, 50
dr1, dr2, dr3, dr4, drEE = 1, 1, 1, 1, 0.5
joint_damping = 1
grav = -9.81
prong_size=0.05
prong_tip_size=-ball_radius
prong_angle=np.pi/4
prong_tip_angle=np.pi/2+prong_angle

# --------------------
# Full Mujoco XML
# --------------------
robot_5dof = f'''
<mujoco model="3dof_arm">
    <compiler angle="radian" coordinate="local" meshdir="assets" autolimits="true"/>
    <option gravity="0 0 {grav}"/>

    <asset>
        <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3" rgb2=".2 .3 .4" width="300" height="300"/>
        <material name="grid" texture="grid" texrepeat="8 8" reflectance=".2"/>
        <mesh name="ee" file="ee1.STL" scale="0.05 0.05 0.05"/>
    </asset>

    <worldbody>
        <geom size="4 4 0.01" type="plane" material="grid"/>
        <light pos="0 0 1"/>

        <!-- Base and Arm -->
        <body name="base" pos="{base_pos_x} {base_pos_y} {base_pos_z}">
            <body name="joint1" pos="0 0 0">
                <joint name="joint1_hinge" type="hinge" axis="0 0 1" damping="{joint_damping}"/>
                <geom type="sphere" size="0.05" rgba="1 0 0 1" contype="1" conaffinity="0"/>
                <geom type="cylinder" fromto="0 0 0 0 0 {l1}" size="0.02" rgba="0 1 0 1" contype="1" conaffinity="0"/>
                
                <body name="joint2" pos="0 0 {l1}" gravcomp="1">
                    <joint name="joint2_hinge" type="hinge" axis="0 1 0" damping="{joint_damping}"/>
                    <geom type="sphere" size="0.05" rgba="1 0 0 1"/>
                    <geom type="cylinder" fromto="0 0 0 0 0 {l2}" size="0.02" rgba="0 1 0 1"/>
                    
                    <body name="joint3" pos="0 0 {l2}" gravcomp="1">
                        <joint name="joint3_hinge" type="hinge" axis="0 1 0" damping="{joint_damping}"/>
                        <geom type="sphere" size="0.05" rgba="1 0 0 1"/>
                        <geom type="cylinder" fromto="0 0 0 0 0 {l3}" size="0.02" rgba="0 1 0 1"/>
                    
                        <body name="joint4" pos="0 0 {l3}" gravcomp="1">
                            <joint name="joint4_hinge" type="hinge" axis="0 1 0" damping="{joint_damping}"/>
                            <geom type="sphere" size="0.05" rgba="1 0 0 1"/>
                            <geom type="cylinder" fromto="0 0 0 0 0 {l4}" size="0.02" rgba="0 1 0 1"/>

                            <body name="ee" pos="0 0 {l4}" gravcomp="1">
                                <joint name="jointEE_hinge" type="hinge" axis="0 1 0" actuatorgravcomp="true"/>
                                <geom type="sphere" size="0.02" rgba="1 1 1 1"/>

                                <!-- prong 0 -->
                                <body name="prong_rot_0" pos="0 0 0" euler="0 0 0">
                                    <body name="prong_0" pos="0 0 0" euler="{prong_angle} 0 0">
                                        <geom  type="capsule" fromto="0 0 0 0 0 {prong_size}" size="0.01" rgba="0.8 0.8 0.8 1"/>
                                        <body name="prong_0_tip" pos="0 0 {prong_size}" euler="{prong_tip_angle} 0 0">
                                            <geom type="capsule" fromto="0 0 0 0 0 {prong_tip_size}" size="0.007" rgba="0.7 0.7 0.7 1"/>
                                        </body>
                                    </body>
                                </body>

                                <!-- prong 1 -->
                                <body name="prong_rot_1" pos="0 0 0" euler="0 0 0.785398">
                                    <body name="prong_1" pos="0 0 0" euler="{prong_angle} 0 0">
                                        <geom  type="capsule" fromto="0 0 0 0 0 {prong_size}" size="0.01" rgba="0.8 0.8 0.8 1"/>
                                        <body name="prong_1_tip" pos="0 0 {prong_size}" euler="{prong_tip_angle} 0 0">
                                            <geom type="capsule" fromto="0 0 0 0 0 {prong_tip_size}" size="0.007" rgba="0.7 0.7 0.7 1"/>
                                        </body>
                                    </body>
                                </body>

                                <!-- prong 2 -->
                                <body name="prong_rot_2" pos="0 0 0" euler="0 0 1.570796">
                                    <body name="prong_2" pos="0 0 0" euler="{prong_angle} 0 0">
                                        <geom type="capsule" fromto="0 0 0 0 0 {prong_size}" size="0.01"/>
                                        <body name="prong_2_tip" pos="0 0 {prong_size}" euler="{prong_tip_angle} 0 0">
                                            <geom type="capsule" fromto="0 0 0 0 0 {prong_tip_size}" size="0.007"/>
                                        </body>
                                    </body>
                                </body>

                                <!-- prong 3 -->
                                <body name="prong_rot_3" pos="0 0 0" euler="0 0 2.356194">
                                    <body name="prong_3" pos="0 0 0" euler="{prong_angle} 0 0">
                                        <geom type="capsule" fromto="0 0 0 0 0 {prong_size}" size="0.01"/>
                                        <body name="prong_3_tip" pos="0 0 {prong_size}" euler="{prong_tip_angle} 0 0">
                                            <geom type="capsule" fromto="0 0 0 0 0 {prong_tip_size}" size="0.007"/>
                                        </body>
                                    </body>
                                </body>

                                <!-- prong 4 -->
                                <body name="prong_rot_4" pos="0 0 0" euler="0 0 3.141592">
                                    <body name="prong_4" pos="0 0 0" euler="{prong_angle} 0 0">
                                        <geom type="capsule" fromto="0 0 0 0 0 {prong_size}" size="0.01"/>
                                        <body name="prong_4_tip" pos="0 0 {prong_size}" euler="{prong_tip_angle} 0 0">
                                            <geom type="capsule" fromto="0 0 0 0 0 {prong_tip_size}" size="0.007"/>
                                        </body>
                                    </body>
                                </body>

                                <!-- prong 5 -->
                                <body name="prong_rot_5" pos="0 0 0" euler="0 0 3.926990">
                                    <body name="prong_5" pos="0 0 0" euler="{prong_angle} 0 0">
                                        <geom type="capsule" fromto="0 0 0 0 0 {prong_size}" size="0.01"/>
                                        <body name="prong_5_tip" pos="0 0 {prong_size}" euler="{prong_tip_angle} 0 0">
                                            <geom type="capsule" fromto="0 0 0 0 0 {prong_tip_size}" size="0.007"/>
                                        </body>
                                    </body>
                                </body>

                                <!-- prong 6 -->
                                <body name="prong_rot_6" pos="0 0 0" euler="0 0 4.712388">
                                    <body name="prong_6" pos="0 0 0" euler="{prong_angle} 0 0">
                                        <geom type="capsule" fromto="0 0 0 0 0 {prong_size}" size="0.01"/>
                                        <body name="prong_6_tip" pos="0 0 {prong_size}" euler="{prong_tip_angle} 0 0">
                                            <geom type="capsule" fromto="0 0 0 0 0 {prong_tip_size}" size="0.007"/>
                                        </body>
                                    </body>
                                </body>

                                <!-- prong 7 -->
                                <body name="prong_rot_7" pos="0 0 0" euler="0 0 5.497787">
                                    <body name="prong_7" pos="0 0 0" euler="{prong_angle} 0 0">
                                        <geom type="capsule" fromto="0 0 0 0 0 {prong_size}" size="0.01"/>
                                        <body name="prong_7_tip" pos="0 0 {prong_size}" euler="{prong_tip_angle} 0 0">
                                            <geom type="capsule" fromto="0 0 0 0 0 {prong_tip_size}" size="0.007"/>
                                        </body>
                                    </body>
                                </body>
                            </body>
                        </body>
                    </body>
                </body>
            </body>
        </body>

        <!-- Ball -->
        <body name="ball" pos="{ball_pos_x} {ball_pos_y} {ball_pos_z}">
            <geom type="sphere" size="{ball_radius}" rgba="1 1 0 1"/>
            <joint type="free"/>
        </body>

        <!-- Maze -->
        {maze_section}

    </worldbody>

    <actuator>
        <position joint="joint1_hinge" kp="{kp1}" dampratio="{dr1}"/>
        <position joint="joint2_hinge" kp="{kp2}" dampratio="{dr2}"/>
        <position joint="joint3_hinge" kp="{kp3}" dampratio="{dr3}"/>
        <position joint="joint4_hinge" kp="{kp4}" dampratio="{dr4}"/>
        <position joint="jointEE_hinge" kp="{kpEE}" dampratio="{drEE}"/>
    </actuator>
</mujoco>
'''

In [ ]:
## Helper functions

def ik_step_jacobian(model, data, goal_pos, damping=0.01, scale=1.0, njoints=4):
    """
    Compute one IK update step using Damped Least Squares (Jacobian pseudo-inverse),
    including orientation control so the end effector always points downward.
    """
    ee_id = model.body('ee').id

    mujoco.mj_forward(model, data)

    ee_pos = data.body(ee_id).xpos.copy()
    pos_error = goal_pos - ee_pos

    # Point end effector down towards ground
    R = data.body(ee_id).xmat.reshape(3, 3)
    ee_z_axis = R[:, 2]
    world_down = np.array([0, 0, -1])
    orient_error = np.cross(ee_z_axis, world_down)

    # Jacobian step
    Jp = np.zeros((3, model.nv))
    Jr = np.zeros((3, model.nv))
    mujoco.mj_jac(model, data, Jp, Jr, ee_pos.copy(), ee_id)
    Jp = Jp[:, :njoints]
    Jr = Jr[:, :njoints]
    w_rot = 0.5   # rotation weight (tune if end effector is not stiff enough
    J_full = np.vstack([Jp, w_rot * Jr])
    err_full = np.hstack([pos_error, w_rot * orient_error])

    # Damped least squares
    JJt = J_full @ J_full.T
    dtheta = J_full.T @ np.linalg.inv(JJt + damping * np.eye(JJt.shape[0])) @ err_full

    return scale * dtheta


def instructions_to_coordinates(instructions, start=np.array([ball_pos_x, ball_pos_y, ball_pos_z]), step_size=cell_size):
    # turns instructions into an array of points
    coordinates = [start]
    z_rise = 0.05  # keep arm raised
    coordinates[0][2] = ball_pos_z + z_rise
    for inst in instructions:
        x, y, z = coordinates[-1].copy()
        if inst == "U": 
            y += step_size
        elif inst == "D": 
            y -= step_size
        elif inst == "R": 
            x += step_size
        elif inst == "L": 
            x -= step_size
        else: 
            raise ValueError(f"Invalid instruction: {inst}")
        coordinates.append(np.array([x, y, ball_pos_z + z_rise]))
    return coordinates

def dist(point1, point2):
    return np.linalg.norm(np.array(point2) - np.array(point1))

In [ ]:
## Main simulation loop

# Change these next two lines based on the model
model = mujoco.MjModel.from_xml_string(robot_4dof)
njoints = 4

data  = mujoco.MjData(model)
ee_id = model.body('ee').id
base_id = model.body('base').id
ball_id = model.body('ball').id

base_xpos = data.body(base_id).xpos.copy()

tol_next_step = 0.05
tol_reset_arm = 0.1
update_freq = 1/1000

start_ball = np.array([ball_pos_x, ball_pos_y, ball_pos_z])
coord_index = 0
current_step = 0
current_tile = (29, 0)
instructions = fogVisualize.generate_and_solve_maze_2(current_tile)
coordinates = instructions_to_coordinates(instructions[0], start=start_ball)
num_collisions = 0
collided = False
ignore_collisions = ["ball", "joint1"]

with mujoco.viewer.launch_passive(model, data) as v:
    while v.is_running():
        for i in range(data.ncon):
            contact = data.contact[i]
            geom1_id = contact.geom1
            geom2_id = contact.geom2
            body1_id = model.geom_bodyid[geom1_id]
            body2_id = model.geom_bodyid[geom2_id]
            body1_name = model.body(body1_id).name
            body2_name = model.body(body2_id).name
            if body1_name not in ignore_collisions and body2_name not in ignore_collisions:
                collided = True

        # current target coordinate
        x_goal, y_goal, z_goal = coordinates[coord_index]
        ee_pos = data.body(ee_id).xpos
        ball_pos = data.body(ball_id).xpos
        ball_to_ee_dist = dist(ball_pos[:2], ee_pos[:2])
        goal = np.array([x_goal, y_goal, z_goal])
        error = dist(ee_pos, goal)
        
        # Nested while loop is horrible style
        #TODO: get rid of this atrocious thing
        while ball_to_ee_dist > tol_reset_arm:
            ball_to_ee_dist = dist(ball_pos[:2], ee_pos[:2])
            goal_reposition = goal.copy()
            goal_reposition[2] = wall_height * 2

            dtheta = ik_step_jacobian(
                        model,
                        data,
                        goal_reposition,
                        damping=0.5,        
                        scale=1.5,
                        njoints=njoints
                    )
            theta_curr = data.qpos[:4].copy()
            theta_cmd = theta_curr + dtheta

            # keep final joint aligned for pointing down
            theta_cmd[-1] = np.pi - np.sum(theta_cmd[1:-1])
            data.ctrl[:] = theta_cmd

            mujoco.mj_step(model, data)
            v.sync()
            time.sleep(update_freq)


        if error < tol_next_step:
            if collided:
                num_collisions += 1
            current_step += 1
            coord_index += 1
            if instructions:  # not empty
                inst = instructions[0]
                collision_rate = (num_collisions / current_step) * 100 if current_step > 0 else 0
                print(f"""Current Step: {current_step},
                          Number of collisions: {num_collisions},
                          Collision rate: {collision_rate}%""")

                if inst == "L":
                    current_tile = (current_tile[0], current_tile[1]-1) 
                elif inst == "R":
                    current_tile = (current_tile[0], current_tile[1]+1)
                elif inst == "U":
                    current_tile = (current_tile[0]-1, current_tile[1])
                elif inst == "D":
                    current_tile = (current_tile[0]+1, current_tile[1])

                # recompute remaining instructions from new tile
                instructions = fogVisualize.generate_and_solve_maze_2(current_tile)
                coordinates = instructions_to_coordinates(instructions[0], start=coordinates[1])
                coord_index = 0
            collided = False

        # IK update
        dtheta = ik_step_jacobian(
            model,
            data,
            goal,
            damping=1,
            scale=1.5,
            njoints=njoints
        )

        theta_curr = data.qpos[:njoints].copy()
        theta_cmd = theta_curr + dtheta

        # keep final joint aligned for pointing down
        theta_cmd[-1] = np.pi - np.sum(theta_cmd[1:-1])
        data.ctrl[:] = theta_cmd

        mujoco.mj_step(model, data)
        v.sync()
        time.sleep(update_freq)